In [ ]:
# TODO: make cells below work from `.csv` file 


In [ ]:
from pathlib import Path
import pandas as pd
import altair as alt

import ideafast_deviceselection as ifds

source = Path(ifds.__file__).parent.parent.absolute() / 'local/keyboard'
users = source /'users'

In [ ]:
# quality of data

quality_by_patient = df.groupby(['participant','quality']).agg({'datetime':['count']})
quality_by_patient.columns = ['count']

quality_total = quality_by_patient.groupby('quality').sum()

print(f'Quality and total of all ({quality_total.sum().sum()}) records  \n')
print(quality_total)

alt.Chart(quality_by_patient.reset_index()).mark_bar().encode(
    x='participant:N',
    y='count',
    color='quality'
)

In [ ]:
# Calculate metrics per day, filter on valid and non-disregarded entries

overview = df[df['valid']=='yes'].groupby(['participant','date_rel']).agg({'wpm': ['mean', 'min', 'max'], 'error_count':['mean', 'min', 'max']})
overview.columns = ['wpm_mean', 'wpm_min', 'wpm_max', 'error_mean', 'error_min', 'error_max']

In [ ]:
base = alt.Chart(overview.reset_index())

selection = alt.selection_multi(fields=['participant'], bind='legend')

base.mark_line().encode(
    alt.X('date_rel:O', axis=alt.Axis(title='Days into the study')),
    alt.Y('wpm_mean:Q', axis=alt.Axis(title='Average words-per-minute')),
    color='participant:N',
    opacity = alt.condition(selection, alt.value(1.0), alt.value(0.05))
).add_selection(
    selection
).properties(
    title='Average words-per-minute, per day'
)
